In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr, ttest_ind

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, explained_variance_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import RFE, SelectKBest, f_regression

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare
import explore
import model

# Goal <br>
The goal of this project is to identify predictive features for the value of properties. <br>
Area (sqft), number of bedrooms, and number of bathrooms are likely to influence property values.

# Acquire

In [2]:
df = acquire.get_zillow_data()
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [3]:
df.shape

(2152863, 7)

In [4]:
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

In [5]:
df.isna().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

# Prepare

Look at tax rates by county.

In [ ]:
tax_info = explore.get_tax_info(df)

In [ ]:
los_angeles_median = tax_info[tax_info.fips == 6037].tax_rates.median()
orange_median = tax_info[tax_info.fips == 6059].tax_rates.median()
ventura_median = tax_info[tax_info.fips == 6111].tax_rates.median()

print("Median tax rate for Los Angelos", los_angeles_median)
print("Median tax rate for Onange", orange_median)
print("Median tax rate for Ventura", ventura_median)

### Tax rates by county <br>
- Los Angelos county has the highest tas rate.
- Ventura and Orange counties have similar tax rates.

In [6]:
zillow = prepare.prepare_zillow(df)
zillow.head()

,bedroom,bathroom,area,taxvalue,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
18,3.0,1.0,1244.0,169471.0,1950.0,2532.88,6037.0
19,3.0,2.0,1300.0,233266.0,1950.0,3110.99,6037.0


In [7]:
# Spit the data into train, validate, and test subsets
train, validate, test = prepare.zillow_split(zillow)
train.shape, validate.shape, test.shape

((1601181, 7), (533728, 7), (426982, 7))

In [8]:
train.head()

,bedroom,bathroom,area,taxvalue,yearbuilt,taxamount,fips
231660,2.0,1.0,810.0,303199.0,1950.0,3740.97,6037.0
205263,4.0,2.0,1875.0,243858.0,1964.0,2993.74,6111.0
350760,3.0,2.0,1768.0,208122.0,1960.0,2834.69,6037.0
283943,3.0,2.0,1527.0,275999.0,1929.0,3661.24,6037.0
292376,3.0,1.0,1191.0,60014.0,1949.0,892.84,6037.0


# Explore

In [ ]:
# Explore the train subset
# The train subset is too big to easily plot it, so I will randomly pick 1000 rows
train_sampled = train.sample(n=1000, axis=0)

sns.pairplot(train_sampled)

In [ ]:
# This tells us which features are correlated with taxvalue
train_sampled.corr()

In [ ]:
# Correlation tests are used to check if two features are correlated
r, p = pearsonr(train.taxvalue, train.area)
r,p

In [ ]:
r, p = pearsonr(train.taxvalue, train.bedroom)
r,p

In [ ]:
r, p = pearsonr(train.taxvalue, train.bathroom)
r,p

In [ ]:
f, p = ttest_ind(train.taxvalue, train.area)
f, p

In [ ]:
f, p = ttest_ind(train.taxvalue, train.bedroom)
f, p

In [ ]:
f, p = ttest_ind(train.taxvalue, train.bathroom)
f, p

# Model & Evaluate

Scale the predictive features, but not the target

In [9]:
# Create subsets with only predictive features (x)
# Create subsets with only target feature (y)
x_train = train.drop(columns=['taxvalue','taxamount'])
y_train = train.taxvalue
x_validate = train.drop(columns=['taxvalue','taxamount'])
y_validate = train.taxvalue
x_test = train.drop(columns=['taxvalue','taxamount'])
y_test = train.taxvalue

In [10]:
x_train_scaled, x_validate_scaled, x_test_scaled = prepare.zillow_scale(x_train, x_validate, x_test)
x_train_scaled.shape, x_validate_scaled.shape, x_test_scaled.shape

((1601181, 5), (1601181, 5), (1601181, 5))

In [11]:
y_train.shape, y_validate.shape, y_test.shape

((1601181,), (1601181,), (1601181,))

### Using Kbest to identify importance of features

In [12]:
# Use Select KBest to identify the best predictors
kbest = model.select_kbest(x_train_scaled, y_train, x_train, 3)
kbest

Index(['bedroom', 'bathroom', 'area'], dtype='object')

### Using RFE to identify importance of features

In [13]:
# Use RFE to identify the best predictors
k=3
rfe_best = model.select_rfe(x_train_scaled, y_train, x_train, k)
rfe_best

Index(['bedroom', 'bathroom', 'area'], dtype='object')

In [14]:
# Convert target series back into pd dataframe
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

y_train.head()

,taxvalue
231660,303199.0
205263,243858.0
350760,208122.0
283943,275999.0
292376,60014.0


### Creating a baseline model

In [ ]:
# Create a mean baseline
# Baseline is the median even though it says mean, so I don't have to make many changes
price_pred_mean = y_train.taxvalue.median()
y_train['price_pred_mean'] = price_pred_mean
y_validate['price_pred_mean'] = price_pred_mean
y_test['price_pred_mean'] = price_pred_mean

y_train.head()

In [ ]:
# RMSE of price_pred_mean (mean baseline)
rmse_train = mean_squared_error(y_train.taxvalue,
                                y_train.price_pred_mean) ** .5

rmse_validate = mean_squared_error(y_validate.taxvalue, 
                                   y_validate.price_pred_mean) ** .5

rmse_test = mean_squared_error(y_test.taxvalue, 
                                   y_validate.price_pred_mean) ** .5

r2_train = explained_variance_score(y_train.taxvalue, y_train.price_pred_mean)
r2_validate = explained_variance_score(y_validate.taxvalue, y_validate.price_pred_mean)


print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2), 
      "\n",
      "\n",
      "R^2 using Mean\nTrain/In-Sample: ", round(r2_train, 2),
      "\nValidate/Out-of-Sample: ", round(r2_validate, 2)
     )


In [ ]:
sns.scatterplot(x=y_train['taxvalue'], y=y_train['price_pred_mean'])

### Creating a linear regression model

In [ ]:
# Linear regression, OLS

# Create the object
lm = LinearRegression(normalize=True)

# Fit the object
lm.fit(x_train_scaled, y_train.taxvalue)

# Use the object
y_train['price_pred_lm'] = lm.predict(x_train)
rmse_train = mean_squared_error(y_train.taxvalue, y_train.price_pred_lm) ** (1/2)

y_validate['price_pred_lm'] = lm.predict(x_validate)
rmse_validate = mean_squared_error(y_validate.taxvalue, y_validate.price_pred_lm) ** (1/2)

r2_train = explained_variance_score(y_train.taxvalue, y_train.price_pred_lm)
r2_validate = explained_variance_score(y_validate.taxvalue, y_validate.price_pred_lm)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", round(rmse_train,2), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate,2),
      "\n",
      "\n",
      "R^2 using Mean\nTrain/In-Sample: ", round(r2_train, 2),
      "\nValidate/Out-of-Sample: ", round(r2_validate, 2))

In [ ]:
sns.scatterplot(x=y_train['taxvalue'], y=y_train['price_pred_lm'])

### Creating a LassoLars model

In [ ]:
# Create the object
lars = LassoLars(alpha=1)

# Fit the model to train. 
# We must specify the column in y_train, 
# because we have converted it to a dataframe from a series!
lars.fit(x_train, y_train.taxvalue)

# predict train
y_train['price_pred_lars'] = lars.predict(x_train)

# evaluate using rmse
rmse_train = mean_squared_error(y_train.taxvalue, y_train.price_pred_lars) ** (1/2)

# predict validate
y_validate['price_pred_lars'] = lars.predict(x_validate)

# evaluate using rmse
rmse_validate = mean_squared_error(y_validate.taxvalue, y_validate.price_pred_lars) ** (1/2)

r2_train = explained_variance_score(y_train.taxvalue, y_train.price_pred_lars)
r2_validate = explained_variance_score(y_validate.taxvalue, y_validate.price_pred_lars)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", round(rmse_train,2), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate,2),
      "\n",
      "\n",
      "R^2 using Mean\nTrain/In-Sample: ", round(r2_train, 2),
      "\nValidate/Out-of-Sample: ", round(r2_validate, 2))

In [ ]:
sns.scatterplot(x=y_train['taxvalue'], y=y_train['price_pred_lars'])

### Creating a generalized linear model (GLM)

In [ ]:
# Create the object
glm = TweedieRegressor(power=1, alpha=0)


# Fit the model to train. 
# We must specify the column in y_train, 
# becuase we  converted it to a dataframe from a series! 
glm.fit(x_train, y_train.taxvalue)

# predict train
y_train['price_pred_glm'] = glm.predict(x_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.taxvalue, y_train.price_pred_glm) ** (1/2)

# predict validate
y_validate['price_pred_glm'] = glm.predict(x_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvalue, y_validate.price_pred_glm) ** (1/2)

r2_train = explained_variance_score(y_train.taxvalue, y_train.price_pred_glm)
r2_validate = explained_variance_score(y_validate.taxvalue, y_validate.price_pred_glm)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", round(rmse_train,2), 
      "\nValidation/Out-of-Sample: ", round(rmse_validate,2),
      "\n",
      "\n",
      "R^2 using Mean\nTrain/In-Sample: ", round(r2_train, 2),
      "\nValidate/Out-of-Sample: ", round(r2_validate, 2))

In [ ]:
sns.scatterplot(x=y_train['taxvalue'], y=y_train['price_pred_glm'])

In [ ]:
sns.scatterplot(x=y_test['taxvalue'], y=y_test['price_pred_lars'])

### LassoLars was the best model

In [ ]:
# predict test
y_test['price_pred_lars'] = lars.predict(x_test)

# evaluate using rmse
rmse_test = mean_squared_error(y_test.taxvalue, y_test.price_pred_lars) ** (1/2)

r2_test = explained_variance_score(y_test.taxvalue, y_test.price_pred_lars)

print("RMSE for Lasso + Lars\n",
      "Test/Out-of-Sample: ", round(rmse_test,2),
      "\n",
      "\n",
      "R^2 using Mean\n",
      "Test/Out-of-Sample: ", round(r2_test, 2))

In [ ]:
sns.scatterplot(x=y_test['taxvalue'], y=y_test['price_pred_lars'])

# Takeaways <br>
- The three most predictive features for tax value are: area, bedroom, bathroom.
- LassaLars was best model as determined by RMSE and R^2.
- County had an effect on tax value, but was not one of the most important drivers.